In [1]:
!git clone -b qa_embedding https://github.com/collaborative-earth/aicacia.git
%cd aicacia/notebooks
from google.colab import drive
drive.mount('/content/drive')

Cloning into 'aicacia'...
remote: Enumerating objects: 1051, done.
remote: Counting objects: 100% (419/419), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 1051 (delta 253), reused 325 (delta 202), pack-reused 632 (from 1)
Receiving objects: 100% (1051/1051), 7.64 MiB | 6.03 MiB/s, done.
Resolving deltas: 100% (498/498), done.
/content/aicacia/notebooks
Mounted at /content/drive


In [2]:
#INstall dependencies
%pip install datasets
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-embeddings-huggingface
%pip install llama-index-finetuning
%pip install llama-index
%pip install llama-index-llms-ollama
%pip install transformers==4.45.2 sentence-transformers==3.1.1
!pip install colab-xterm
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.6 MB/s eta 0:00:00
   ━━━

In [3]:
import json
import os
import sys
import random
import torch
sys.path.append("..")
from pathlib import Path
# llama index packages
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.finetuning import SentenceTransformersFinetuneEngine
# Sentence transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from finetuning.src.utils import split_dataset
os.environ["WANDB_MODE"] = "disabled"


In [4]:
"""Sentence Transformer Finetuning Engine."""

from typing import Any, Optional

from llama_index.core.base.embeddings.base import BaseEmbedding
from llama_index.core.embeddings.utils import resolve_embed_model
from llama_index.finetuning.embeddings.common import (
    EmbeddingQAFinetuneDataset,
)
from llama_index.finetuning.types import BaseEmbeddingFinetuneEngine
from sentence_transformers import InputExample, SentenceTransformer, datasets, losses

class MySentenceTransformersFinetuneEngine(BaseEmbeddingFinetuneEngine):
    """Sentence Transformers Finetune Engine."""

    def __init__(
        self,
        dataset: EmbeddingQAFinetuneDataset,
        model_id: str = "BAAI/bge-small-en",
        model_output_path: str = "exp_finetune",
        batch_size: int = 10,
        val_dataset: Optional[EmbeddingQAFinetuneDataset] = None,
        loss: Optional[Any] = None,
        epochs: int = 2,
        show_progress_bar: bool = True,
        evaluation_steps: int = 50,
        use_all_docs: bool = False,
        trust_remote_code: bool = False,
        device: Optional[Any] = None,
    ) -> None:
        """Init params."""



        self.dataset = dataset

        self.model_id = model_id
        self.model_output_path = model_output_path
        self.model = SentenceTransformer(
            model_id, trust_remote_code=trust_remote_code, device=device
        )

        self.use_all_docs = use_all_docs

        examples: Any = []
        for query_id, query in dataset.queries.items():
            if use_all_docs:
                for node_id in dataset.relevant_docs[query_id]:
                    text = dataset.corpus[node_id]
                    example = InputExample(texts=[query, text])
                    examples.append(example)
            else:
                node_id = dataset.relevant_docs[query_id][0]
                text = dataset.corpus[node_id]
                example = InputExample(texts=[query, text])
                examples.append(example)

        self.examples = examples

        # Remove duplicated questions
        self.loader =datasets.NoDuplicatesDataLoader(examples, batch_size=batch_size)
        #self.loader: DataLoader = datasets.NoDuplicatesDataLoader(examples, batch_size=batch_size

        # define evaluator
        from sentence_transformers.evaluation import InformationRetrievalEvaluator

        evaluator: Optional[InformationRetrievalEvaluator] = None
        if val_dataset is not None:
            evaluator = InformationRetrievalEvaluator(
                val_dataset.queries, val_dataset.corpus, val_dataset.relevant_docs
            )
        self.evaluator = evaluator

        # define loss
        self.loss = loss or losses.MultipleNegativesRankingLoss(self.model)

        self.epochs = epochs
        self.show_progress_bar = show_progress_bar
        self.evaluation_steps = evaluation_steps
        self.warmup_steps = int(len(self.loader) * epochs * 0.1)

    def finetune(self, **train_kwargs: Any) -> None:
        """Finetune model."""
        self.model.fit(
            train_objectives=[(self.loader, self.loss)],
            epochs=self.epochs,
            warmup_steps=self.warmup_steps,
            output_path=self.model_output_path,
            show_progress_bar=self.show_progress_bar,
            evaluator=self.evaluator,
            evaluation_steps=self.evaluation_steps,
        )

    def get_finetuned_model(self, **model_kwargs: Any) -> BaseEmbedding:
        """Gets finetuned model."""
        embed_model_str = "local:" + self.model_output_path
        return resolve_embed_model(embed_model_str)

In [5]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512,expandable_segments:True"
# Enable gradient checkpointing to reduce memory usage
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [6]:
QADATASET_PATH = "/content/drive/Shareddrives/Aicacia MicroLab/QA_datasets/qa_finetune_dataset_marcosT5.json"
qa_dataset = EmbeddingQAFinetuneDataset.from_json(QADATASET_PATH)
train_dataset,val_dataset =split_dataset(qa_dataset,0.7)

In [ ]:
#model = SentenceTransformer("jinaai/jina-embeddings-v3",trust_remote_code=True)

In [ ]:
# TODO: use train function from sentence transformer to have a more fine grained control over the parameters
finetune_engine = MySentenceTransformersFinetuneEngine(
    train_dataset,
    model_id= "BAAI/bge-m3",
    model_output_path="test_model",
    val_dataset = val_dataset,
    evaluation_steps= 500,
    trust_remote_code = True,
    use_all_docs = True,
    batch_size = 2,
    device = "cuda"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
finetune_engine.finetune()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100,Dot Accuracy@1,Dot Accuracy@3,Dot Accuracy@5,Dot Accuracy@10,Dot Precision@1,Dot Precision@3,Dot Precision@5,Dot Precision@10,Dot Recall@1,Dot Recall@3,Dot Recall@5,Dot Recall@10,Dot Ndcg@10,Dot Mrr@10,Dot Map@100
500,0.070800,No log,0.261202,0.422951,0.496175,0.593443,0.261202,0.140984,0.099235,0.059344,0.261202,0.422951,0.496175,0.593443,0.415372,0.359730,0.371502,0.261202,0.422951,0.496175,0.593443,0.261202,0.140984,0.099235,0.059344,0.261202,0.422951,0.496175,0.593443,0.415372,0.359730,0.371502


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100,Dot Accuracy@1,Dot Accuracy@3,Dot Accuracy@5,Dot Accuracy@10,Dot Precision@1,Dot Precision@3,Dot Precision@5,Dot Precision@10,Dot Recall@1,Dot Recall@3,Dot Recall@5,Dot Recall@10,Dot Ndcg@10,Dot Mrr@10,Dot Map@100
500,0.070800,No log,0.261202,0.422951,0.496175,0.593443,0.261202,0.140984,0.099235,0.059344,0.261202,0.422951,0.496175,0.593443,0.415372,0.359730,0.371502,0.261202,0.422951,0.496175,0.593443,0.261202,0.140984,0.099235,0.059344,0.261202,0.422951,0.496175,0.593443,0.415372,0.359730,0.371502
1000,0.091600,No log,0.272131,0.438251,0.512568,0.609836,0.272131,0.146084,0.102514,0.060984,0.272131,0.438251,0.512568,0.609836,0.432782,0.377117,0.388515,0.272131,0.438251,0.512568,0.609836,0.272131,0.146084,0.102514,0.060984,0.272131,0.438251,0.512568,0.609836,0.432782,0.377117,0.388515
1066,0.091600,No log,0.268852,0.455738,0.514754,0.620765,0.268852,0.151913,0.102951,0.062077,0.268852,0.455738,0.514754,0.620765,0.435873,0.377832,0.390116,0.268852,0.455738,0.514754,0.620765,0.268852,0.151913,0.102951,0.062077,0.268852,0.455738,0.514754,0.620765,0.435873,0.377832,0.390116
1500,0.026600,No log,0.288525,0.445902,0.527869,0.640437,0.288525,0.148634,0.105574,0.064044,0.288525,0.445902,0.527869,0.640437,0.450996,0.392086,0.403566,0.288525,0.445902,0.527869,0.640437,0.288525,0.148634,0.105574,0.064044,0.288525,0.445902,0.527869,0.640437,0.450996,0.392086,0.403566
2000,0.005000,No log,0.301639,0.478689,0.556284,0.651366,0.301639,0.159563,0.111257,0.065137,0.301639,0.478689,0.556284,0.651366,0.466188,0.408014,0.419770,0.301639,0.478689,0.556284,0.651366,0.301639,0.159563,0.111257,0.065137,0.301639,0.478689,0.556284,0.651366,0.466188,0.408014,0.419770
2132,0.005000,No log,0.301639,0.480874,0.557377,0.657923,0.301639,0.160291,0.111475,0.065792,0.301639,0.480874,0.557377,0.657923,0.468877,0.409671,0.420989,0.301639,0.480874,0.557377,0.657923,0.301639,0.160291,0.111475,0.065792,0.301639,0.480874,0.557377,0.657923,0.468877,0.409671,0.420989


# Test trained model

In [ ]:
 !zip -r test_model.zip test_model/

  adding: test_model/ (stored 0%)
  adding: test_model/sentence_bert_config.json (deflated 4%)
  adding: test_model/tokenizer_config.json (deflated 76%)
  adding: test_model/config.json (deflated 48%)
  adding: test_model/2_Normalize/ (stored 0%)
  adding: test_model/config_sentence_transformers.json (deflated 35%)
  adding: test_model/model.safetensors (deflated 21%)
  adding: test_model/tokenizer.json (deflated 76%)
  adding: test_model/sentencepiece.bpe.model (deflated 49%)
  adding: test_model/modules.json (deflated 62%)
  adding: test_model/1_Pooling/ (stored 0%)
  adding: test_model/1_Pooling/config.json (deflated 57%)
  adding: test_model/special_tokens_map.json (deflated 85%)
  adding: test_model/README.md (deflated 70%)


In [9]:
#embed_model_ft = finetune_engine.get_finetuned_model()
TEST_MODEL_ZIP = "/content/drive/Shareddrives/Aicacia MicroLab/finetuning/test_model_2epochs.zip"
OUTPUT_DIR = "/content/drive/Shareddrives/Aicacia MicroLab/finetuning/"
if os.path.exists(TEST_MODEL_ZIP):
    with zipfile.ZipFile(TEST_MODEL_ZIP, 'r') as zip_ref:
        zip_ref.extractall(OUTPUT_DIR)
    print("Unzipping completed successfully!")
else:
    print("File does not exist.")
TEST_MODEL_PATH  = "/content/drive/Shareddrives/Aicacia MicroLab/finetuning/test_model"
embed_model_ft = SentenceTransformer(TEST_MODEL_PATH,trust_remote_code=True)

ValueError: Path /content/drive/Shareddrives/Aicacia MicroLab/finetuning/test_model_2epochs not found

In [15]:
TEST_MODEL_PATH  = "/content/drive/Shareddrives/Aicacia MicroLab/finetuning/test_model"
embed_model_ft = SentenceTransformer(TEST_MODEL_PATH,trust_remote_code=True)

In [16]:
evaluator = InformationRetrievalEvaluator(
                val_dataset.queries, val_dataset.corpus, val_dataset.relevant_docs,
                name='finetuned',show_progress_bar=True
    )
with torch.no_grad():
  results_ft = evaluator(embed_model_ft,output_path=".")
  print(results_ft)

In [18]:
evaluator2 = InformationRetrievalEvaluator(
                val_dataset.queries, val_dataset.corpus, val_dataset.relevant_docs,
                name='original',show_progress_bar=True
    )
model = SentenceTransformer("BAAI/bge-m3",trust_remote_code=True)
with torch.no_grad():
  results_original = evaluator2(model,output_path=".")
  print(results_original)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [01:58<00:00, 118.60s/it]

{'finetuned_cosine_accuracy@1': 0.38907103825136613, 'finetuned_cosine_accuracy@3': 0.594535519125683, 'finetuned_cosine_accuracy@5': 0.6688524590163935, 'finetuned_cosine_accuracy@10': 0.7475409836065574, 'finetuned_cosine_precision@1': 0.38907103825136613, 'finetuned_cosine_precision@3': 0.19817850637522766, 'finetuned_cosine_precision@5': 0.13377049180327868, 'finetuned_cosine_precision@10': 0.07475409836065573, 'finetuned_cosine_recall@1': 0.38907103825136613, 'finetuned_cosine_recall@3': 0.594535519125683, 'finetuned_cosine_recall@5': 0.6688524590163935, 'finetuned_cosine_recall@10': 0.7475409836065574, 'finetuned_cosine_ndcg@10': 0.5643336347609389, 'finetuned_cosine_mrr@10': 0.5060135310954978, 'finetuned_cosine_map@100': 0.5146065469921121, 'finetuned_dot_accuracy@1': 0.38907103825136613, 'finetuned_dot_accuracy@3': 0.594535519125683, 'finetuned_dot_accuracy@5': 0.6688524590163935, 'finetuned_dot_accuracy@10': 0.7475409836065574, 'finetuned_dot_precision@1': 0.38907103825136613

In [19]:
results_original

{'original_cosine_accuracy@1': 0.3289617486338798,
 'original_cosine_accuracy@3': 0.5060109289617486,
 'original_cosine_accuracy@5': 0.5879781420765028,
 'original_cosine_accuracy@10': 0.687431693989071,
 'original_cosine_precision@1': 0.3289617486338798,
 'original_cosine_precision@3': 0.1686703096539162,
 'original_cosine_precision@5': 0.11759562841530054,
 'original_cosine_precision@10': 0.0687431693989071,
 'original_cosine_recall@1': 0.3289617486338798,
 'original_cosine_recall@3': 0.5060109289617486,
 'original_cosine_recall@5': 0.5879781420765028,
 'original_cosine_recall@10': 0.687431693989071,
 'original_cosine_ndcg@10': 0.49823231810257357,
 'original_cosine_mrr@10': 0.4387639864689041,
 'original_cosine_map@100': 0.4492777423424018,
 'original_dot_accuracy@1': 0.3289617486338798,
 'original_dot_accuracy@3': 0.5060109289617486,
 'original_dot_accuracy@5': 0.5879781420765028,
 'original_dot_accuracy@10': 0.687431693989071,
 'original_dot_precision@1': 0.3289617486338798,
 'ori

In [20]:
results_ft

{'finetuned_cosine_accuracy@1': 0.38907103825136613,
 'finetuned_cosine_accuracy@3': 0.594535519125683,
 'finetuned_cosine_accuracy@5': 0.6688524590163935,
 'finetuned_cosine_accuracy@10': 0.7475409836065574,
 'finetuned_cosine_precision@1': 0.38907103825136613,
 'finetuned_cosine_precision@3': 0.19817850637522766,
 'finetuned_cosine_precision@5': 0.13377049180327868,
 'finetuned_cosine_precision@10': 0.07475409836065573,
 'finetuned_cosine_recall@1': 0.38907103825136613,
 'finetuned_cosine_recall@3': 0.594535519125683,
 'finetuned_cosine_recall@5': 0.6688524590163935,
 'finetuned_cosine_recall@10': 0.7475409836065574,
 'finetuned_cosine_ndcg@10': 0.5643336347609389,
 'finetuned_cosine_mrr@10': 0.5060135310954978,
 'finetuned_cosine_map@100': 0.5146065469921121,
 'finetuned_dot_accuracy@1': 0.38907103825136613,
 'finetuned_dot_accuracy@3': 0.594535519125683,
 'finetuned_dot_accuracy@5': 0.6688524590163935,
 'finetuned_dot_accuracy@10': 0.7475409836065574,
 'finetuned_dot_precision@1': 

In [ ]:
## COde to clear gpu memory issues
import torch

torch.cuda.empty_cache()
import gc

torch.cuda.empty_cache()

In [ ]:
def reset_gpu_memory():
    # Move all variables to CPU
    gc.collect()
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                obj = obj.cpu()
        except:
            pass

    # Clear CUDA cache
    torch.cuda.empty_cache()

In [ ]:
def print_gpu_memory():
    print(f"Current memory allocated: {torch.cuda.memory_allocated()/1024**2:.2f}MB")
    print(f"Max memory allocated: {torch.cuda.max_memory_allocated()/1024**2:.2f}MB")
    print(f"Current memory cached: {torch.cuda.memory_reserved()/1024**2:.2f}MB")

In [ ]:
reset_gpu_memory()
print_gpu_memory()